# Universality: U3 and CNOT decomposition
In the explorer universality challenge we introduced the concept of universality but we focused only on a single qubit. Today we will make things more difficult. It has been proven that with the parametric gate U3 and the CNOT gate we can build any operator. In this exercise, we will be asked to work with the following matrix:

![](images/matrix.png)

You are asked to create an operator in which only U3 and CNOT gates are used and whose associated matrix is the above. Remember that to check the matrix of an operator you can use the `qml.matrix` function. Let's see an example:

```python
def circuit():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 1)
    

>> qml.matrix(circuit)()

array([[ 0.        ,  0.        ,  0.70710678,  0.70710678],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678],
       [ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ]])
```
To evaluate your solution, it will be checked that the matrices match and that the gates used are the ones allowed.

## Challenge code
You must complete the `circuit()` function below. Inside the function you simply have to include the necessary operations to generate the requested matrix.

## Inputs and Outputs
In this challenge, we will not have inputs and outputs. The test will be done by checking that the matrix of your circuit coincides with that of the statement. When testing yourself, you can always print the matrix using `qml.matrix(circuit, wire_order=[0,1,2])`.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `1e-05` relative error tolerance), the output will be `"Correct!"`. Otherwise, you will receive a `"Wrong answer"` prompt.

In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

def L():
    qml.U3(theta=0,phi=0,delta=0, wires=0) # identity
    qml.U3(theta=np.pi,phi=0,delta=np.pi, wires=1) # PauliX

def K():
    qml.CNOT(wires=[0,1])
    L()

def U_tilde():
    qml.U3(theta=-2*np.pi,phi=0,delta=0, wires=0) # U3-
    qml.U3(theta=np.pi/2,phi=0,delta=np.pi, wires=1) # U3*

def F():
    U_tilde()
    K()
    U_tilde()

def circuit():
    """
    Succession of gates that will generate the requested matrix.
    This function does not receive any arguments nor does it return any values.
    """
    F()
    qml.U3(theta=np.pi/2,phi=0,delta=np.pi, wires=2)


# These functions are responsible for testing the solution.

def run(input: str) -> str:
    matrix = qml.matrix(circuit)().real

    with qml.tape.QuantumTape() as tape:
        circuit()

    names = [op.name for op in tape.operations]
    return json.dumps({"matrix": matrix.tolist(), "gates": names})

def check(user_output: str, expected_output: str) -> str:
    parsed_output = json.loads(user_output)
    matrix_user = np.array(parsed_output["matrix"])
    gates = parsed_output["gates"]

    solution = (
        1
        / np.sqrt(2)
        * np.array(
            [
                [1, 1, 0, 0, 0, 0, 0, 0],
                [1, -1, 0, 0, 0, 0, 0, 0],
                [0, 0, -1, -1, 0, 0, 0, 0],
                [0, 0, -1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 0, 0],
                [0, 0, 0, 0, 1, -1, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 1],
                [0, 0, 0, 0, 0, 0, 1, -1],
            ]
        )
    )

    assert np.allclose(matrix_user, solution)
    assert len(set(gates)) == 2 and "U3" in gates and "CNOT" in gates


test_cases = [['No input', 'No output']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        message = 0
        if message == check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
